In [11]:
import gym
import random
import numpy as np
import time
from gym.envs.registration import register
from IPython.display import clear_output

In [12]:
try:
    register(
        id='FrozenLakeNoSlip-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4', 'is_slippery':False},
        max_episode_steps=100,
        reward_threshold=0.78, # optimum = .8196
    )
except:
    pass

# env_name = "CartPole-v1"
# env_name = "MountainCar-v0"
# env_name = "MountainCarContinuous-v0"
# env_name = "Acrobot-v1"
# env_name = "Pendulum-v0"
env_name = "FrozenLake-v0"
env_name = "FrozenLakeNoSlip-v0"
env = gym.make(env_name)
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)
type(env.action_space)

Observation space: Discrete(16)
Action space: Discrete(4)


gym.spaces.discrete.Discrete

In [13]:
class Agent():
    def __init__(self, env):
        self.is_discrete = \
            type(env.action_space) == gym.spaces.discrete.Discrete
        
        if self.is_discrete:
            self.action_size = env.action_space.n
            print("Action size:", self.action_size)
        else:
            self.action_low = env.action_space.low
            self.action_high = env.action_space.high
            self.action_shape = env.action_space.shape
            print("Action range:", self.action_low, self.action_high)
        
    def get_action(self, state):
        if self.is_discrete:
            action = random.choice(range(self.action_size))
        else:
            action = np.random.uniform(self.action_low,
                                       self.action_high,
                                       self.action_shape)
        return action

In [14]:
class QAgent(Agent):
    def __init__(self, env, discount_rate=0.97, learning_rate=0.01):
        super().__init__(env)
        self.state_size = env.observation_space.n
        print("State size:", self.state_size)
        
        self.eps = 1.0
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        self.build_model()
        
    def build_model(self):
        self.q_table = 1e-4*np.random.random([self.state_size, self.action_size])
        
    def get_action(self, state):
        q_state = self.q_table[state]
        action_greedy = np.argmax(q_state)
        action_random = super().get_action(state)
        return action_random if random.random() < self.eps else action_greedy
    
    def train(self, experience):
        state, action, next_state, reward, done = experience
        
        q_next = self.q_table[next_state]
        q_next = np.zeros([self.action_size]) if done else q_next
        q_target = reward + self.discount_rate * np.max(q_next)
        
        q_update = q_target - self.q_table[state,action]
        self.q_table[state,action] += self.learning_rate * q_update
        
        if done:
            self.eps = self.eps * 0.99
        
agent = QAgent(env)

Action size: 4
State size: 16


In [19]:

total_reward = 0
for ep in range(100):
    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        agent.train((state,action,next_state,reward,done))
        state = next_state
        total_reward += reward
        
        print("s:", state, "a:", action)
        print("Episode: {}, Total reward: {}, eps: {}".format(ep,total_reward,agent.eps))
        env.render()
        print(agent.q_table)
        time.sleep(0.005)
        clear_output(wait=True)

s: 15 a: 2
Episode: 99, Total reward: 99.0, eps: 0.010751591703479106
  (Right)
SFFF
FHFH
FFFH
HFFG
[[3.61344588e-05 2.18954979e-02 5.35102316e-05 3.56783793e-05]
 [1.10739798e-04 5.45341823e-05 3.36689169e-05 6.14276426e-05]
 [4.58234349e-05 7.07277899e-05 4.34967363e-05 8.44045816e-05]
 [2.42256893e-05 5.68358344e-06 5.78386921e-05 2.19539617e-05]
 [5.93510583e-05 6.59323501e-02 1.55340779e-05 5.87789729e-05]
 [4.72413242e-05 7.85861243e-05 5.02005180e-05 6.90405354e-05]
 [3.65407805e-06 6.02124467e-05 1.16262872e-05 1.93878044e-05]
 [3.36192568e-05 4.35789065e-05 7.65339570e-05 7.37483739e-05]
 [7.63451937e-04 2.13953274e-05 1.69322243e-01 5.87217421e-05]
 [7.79682322e-04 3.62077365e-01 1.21823736e-04 3.82795105e-05]
 [5.04570949e-05 3.12375794e-02 7.17290512e-05 1.83289450e-05]
 [9.22975843e-05 8.06887598e-05 7.78637111e-05 5.19208803e-05]
 [8.73820287e-07 1.27608926e-05 6.52168954e-05 5.78165940e-05]
 [3.89588849e-05 7.38223234e-05 6.34352406e-01 8.02177642e-05]
 [1.71433427e-03 1